In [26]:
import xgboost
import joblib
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from quadratic_weighted_kappa import quadratic_weighted_kappa

In [27]:
x = joblib.load('essay_sbert_asap7_new')
y = joblib.load("score_asap7")

In [28]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
print(kf)

KFold(n_splits=5, random_state=42, shuffle=True)


In [29]:
model2 = xgboost.XGBRegressor(objective ='reg:squarederror',
                colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.05,
                 max_depth=2,
                 min_child_weight=1.5,
                 n_estimators=1000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)

In [30]:
qwk_scores = []
test_indices = []
pred_labels = []


for train_index, test_index in kf.split(x):
    
    X_train, X_test, Y_train, Y_test = x[train_index], x[test_index], y[train_index], y[test_index]
         
    model2.fit(X_train, Y_train)
    
    predict = model2.predict(X_test)
    predict = np.round(predict, 3)
    
    pred_labels.extend(predict)
    test_indices.extend(test_index)

    result = quadratic_weighted_kappa(Y_test, predict)
    
    print("Qwk : ", result)
    
    qwk_scores.append(result)
    
np.mean(qwk_scores)

Qwk :  0.7132270709511283
Qwk :  0.7173498550570507
Qwk :  0.7619546298529407
Qwk :  0.7142133531179213
Qwk :  0.6966435686948669


0.7206776955347817

In [ ]:
pred_labels_int = list(map(int, pred_labels))

In [31]:
#new_score = np.zeros(1569, dtype=np.int64)
new_score = np.zeros(1569)

In [32]:
#new_score[test_indices] = pred_labels_int
new_score[test_indices] = pred_labels

In [20]:
joblib.dump(new_score, 'score_model_gb_normalized_ngram_sbert')

['score_model_rr_normalized_ngram']

In [33]:
quadratic_weighted_kappa(new_score, y)

0.7220764450004387

In [34]:
# Save the original score in floating number
joblib.dump(new_score, 'score_model_gb_normalized_float_sbert')

['score_model_gb_normalized_float_sbert']

In [35]:
new_score

array([11.56200027, 15.31000042, 13.24400043, ..., 19.90699959,
       20.4279995 , 13.85700035])